In [25]:
import pandas as pd
pd.options.display.precision=2
df = pd.read_csv("USDJPY.csv")

In [26]:
#df = df.query("Date>='1997-01-02'").reset_index()
df = df.query("Date>='1997-01-01'")
#df_today = df.query("Date=='2021-04-30'")
#df = df.query("Date<'2021-04-30'")

In [27]:
df["前日比"] = df["Close"].pct_change()*100
#df = df.sort_values('Date', ascending=False).reset_index()
df = df.reset_index()
df

,index,Date,Open,High,Low,Close,前日比
0,6561,1997-01-02,115.85,116.20,115.28,115.60,NaN
1,6562,1997-01-03,115.65,116.70,115.35,116.28,0.59
2,6563,1997-01-06,116.75,116.98,115.38,115.75,-0.46
3,6564,1997-01-07,115.74,116.02,114.80,115.25,-0.43
4,6565,1997-01-08,115.22,116.00,114.90,115.73,0.42
...,...,...,...,...,...,...,...
6309,12870,2021-04-27,108.09,108.78,108.09,108.72,0.58
6310,12871,2021-04-28,108.71,109.08,108.56,108.60,-0.11
6311,12872,2021-04-29,108.60,109.22,108.44,108.94,0.31
6312,12873,2021-04-30,108.94,109.34,108.72,109.29,0.32


In [28]:
import talib as ta
import numpy as np

close = np.array(df["Close"])

#特徴量を入れるための空のdataframeを作成
df_feature = pd.DataFrame(index=range(len(df)),columns=["SMA5/current", "SMA20/current","RSI","MACD","BBANDS+2σ","BBANDS-2σ"])

#以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる

#単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
df_feature["SMA5/current"]= ta.SMA(close, timeperiod=5) / close
df_feature["SMA20/current"]= ta.SMA(close, timeperiod=20) / close

#RSI
df_feature["RSI"] = ta.RSI(close, timeperiod=14)

#MACD
df_feature["MACD"], _ , _= ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

#ボリンジャーバンド 
upper, middle, lower = ta.BBANDS(close, timeperiod=20, nbdevup=3, nbdevdn=3)
df_feature["BBANDS+2σ"] = upper / close
df_feature["BBANDS-2σ"] = lower / close

In [29]:
df

,index,Date,Open,High,Low,Close,前日比
0,6561,1997-01-02,115.85,116.20,115.28,115.60,NaN
1,6562,1997-01-03,115.65,116.70,115.35,116.28,0.59
2,6563,1997-01-06,116.75,116.98,115.38,115.75,-0.46
3,6564,1997-01-07,115.74,116.02,114.80,115.25,-0.43
4,6565,1997-01-08,115.22,116.00,114.90,115.73,0.42
...,...,...,...,...,...,...,...
6309,12870,2021-04-27,108.09,108.78,108.09,108.72,0.58
6310,12871,2021-04-28,108.71,109.08,108.56,108.60,-0.11
6311,12872,2021-04-29,108.60,109.22,108.44,108.94,0.31
6312,12873,2021-04-30,108.94,109.34,108.72,109.29,0.32


In [30]:
df_feature

,SMA5/current,SMA20/current,RSI,MACD,BBANDS+2σ,BBANDS-2σ
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,1.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
6309,0.99,1.0,50.92,-0.15,1.03,0.98
6310,1.00,1.0,49.17,-0.13,1.03,0.98
6311,1.00,1.0,53.97,-0.09,1.02,0.98
6312,0.99,1.0,58.42,-0.03,1.01,0.98


In [31]:
def classify(x):
#前日比が-0.2%以下ならグループ０
    if x <= -0.2:
        return 0
#前日比が0.2%<x<0.2%ならグループ1
    elif -0.2 < x < 0.2:
        return 1
#前日比が0.2%以上ならグループ２
    elif 0.2 <= x:
        return 2

In [32]:
# df["前日比_float"] = df["前日比%"].apply(lambda x: float(x.replace("%", "")))

#前日比%の分類の仕方。できるだけ各クラスのサンプルが等しいようにわける
def classify(x):
    if x <= -0.2:
        return 0
    elif -0.2 < x < 0.2:
        return 1
    elif 0.2 <= x:
        return 2


df["前日比_classified"] = df["前日比"].apply(lambda x: classify(x))

#教師にしたいデータを一日ずつずらす
df_y = df["前日比_classified"].shift(-1)
df["df_y"] = df["前日比_classified"].shift(-1)

In [33]:
df.head(30)

,index,Date,Open,High,Low,Close,前日比,前日比_classified,df_y
0,6561,1997-01-02,115.85,116.20,115.28,115.60,NaN,NaN,2.0
1,6562,1997-01-03,115.65,116.70,115.35,116.28,5.88e-01,2.0,0.0
2,6563,1997-01-06,116.75,116.98,115.38,115.75,-4.56e-01,0.0,0.0
3,6564,1997-01-07,115.74,116.02,114.80,115.25,-4.32e-01,0.0,2.0
4,6565,1997-01-08,115.22,116.00,114.90,115.73,4.16e-01,2.0,2.0
5,6566,1997-01-09,115.83,116.45,115.65,116.36,5.44e-01,2.0,0.0
6,6567,1997-01-10,116.37,116.67,115.24,116.05,-2.66e-01,0.0,2.0
7,6568,1997-01-13,116.05,116.54,115.85,116.45,3.45e-01,2.0,2.0
8,6569,1997-01-14,116.21,117.04,116.20,116.86,3.52e-01,2.0,1.0
9,6570,1997-01-15,116.93,117.48,116.70,116.85,-8.56e-03,1.0,0.0


In [34]:
df_xy = pd.concat([df_feature, df_y], axis=1)
df_xy = df_xy.dropna(how="any")

In [35]:
pd.concat([df_feature, df_y,df["Close"]], axis=1).tail(30)

,SMA5/current,SMA20/current,RSI,MACD,BBANDS+2σ,BBANDS-2σ,前日比_classified,Close
6284,1.00,0.99,65.86,0.88,1.02,0.97,1.0,108.62
6285,1.00,0.99,67.19,0.83,1.02,0.97,2.0,108.75
6286,1.00,0.99,71.03,0.82,1.02,0.97,2.0,109.16
6287,0.99,0.99,74.81,0.83,1.01,0.97,1.0,109.66
6288,0.99,0.99,75.95,0.85,1.01,0.97,2.0,109.82
6289,0.99,0.99,79.25,0.90,1.00,0.97,2.0,110.36
6290,0.99,0.98,81.16,0.96,1.00,0.97,1.0,110.73
6291,1.00,0.99,78.31,0.98,1.01,0.97,1.0,110.59
6292,1.00,0.99,78.81,0.99,1.01,0.97,0.0,110.68
6293,1.00,0.99,68.27,0.95,1.01,0.97,0.0,110.15


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import optuna

X_train, X_test, Y_train, Y_test = train_test_split(df_xy[["SMA5/current", "SMA20/current","RSI","MACD","BBANDS+2σ","BBANDS-2σ"]],df_xy["前日比_classified"], train_size=0.8,shuffle=False)

def objective(trial):
    min_samples_split = trial.suggest_int("min_samples_split", 2,16)
    max_leaf_nodes = int(trial.suggest_discrete_uniform("max_leaf_nodes", 4,64,4))
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    n_estimators = int(trial.suggest_discrete_uniform("n_estimators", 50,500,50))
    max_depth = trial.suggest_int("max_depth", 3,10)
    clf = RandomForestClassifier(random_state=1, n_estimators = n_estimators, max_leaf_nodes = max_leaf_nodes, max_depth=max_depth, max_features=None,criterion=criterion,min_samples_split=min_samples_split)
    clf.fit(X_train, Y_train)
    return 1 - accuracy_score(Y_test, clf.predict(X_test))

study = optuna.create_study(direction='maximize')
#study.optimize(objective, n_trials=100)
study.optimize(objective, n_trials=5)


[I 2021-05-05 18:41:09,140] A new study created in memory with name: no-name-18623c98-ff7b-4521-bdd9-ea618c812830
[I 2021-05-05 18:41:11,829] Trial 0 finished with value: 0.6035031847133758 and parameters: {'min_samples_split': 15, 'max_leaf_nodes': 4.0, 'criterion': 'gini', 'n_estimators': 50.0, 'max_depth': 10}. Best is trial 0 with value: 0.6035031847133758.
[I 2021-05-05 18:41:25,494] Trial 1 finished with value: 0.6106687898089171 and parameters: {'min_samples_split': 4, 'max_leaf_nodes': 4.0, 'criterion': 'entropy', 'n_estimators': 150.0, 'max_depth': 10}. Best is trial 1 with value: 0.6106687898089171.
[I 2021-05-05 18:41:30,025] Trial 2 finished with value: 0.6090764331210191 and parameters: {'min_samples_split': 5, 'max_leaf_nodes': 8.0, 'criterion': 'entropy', 'n_estimators': 50.0, 'max_depth': 9}. Best is trial 1 with value: 0.6106687898089171.
[I 2021-05-05 18:41:44,613] Trial 3 finished with value: 0.6074840764331211 and parameters: {'min_samples_split': 12, 'max_leaf_node

In [37]:
#ベストな結果、パラメータの表示
print('Accuracy: {}'.format(study.best_trial.value))
print("Best hyperparameters: {}".format(study.best_trial.params))

Accuracy: 0.6106687898089171
Best hyperparameters: {'min_samples_split': 4, 'max_leaf_nodes': 4.0, 'criterion': 'entropy', 'n_estimators': 150.0, 'max_depth': 10}


In [38]:
clf = RandomForestClassifier(max_depth=30, n_estimators=30, random_state=42)
clf.fit(X_train, Y_train)

RandomForestClassifier(max_depth=30, n_estimators=30, random_state=42)

In [39]:
Y_pred = clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.3630573248407643


In [40]:
# チューニングしたハイパーパラメーターをフィット
optimised_rf = RandomForestClassifier(random_state=1, n_estimators = int(study.best_params['n_estimators']), max_leaf_nodes = int(study.best_params['max_leaf_nodes']), max_depth=study.best_params['max_depth'], max_features=None,criterion=study.best_params['criterion'],min_samples_split=study.best_params['min_samples_split'])

In [41]:
optimised_rf.fit(X_train, Y_train)
Y_pred = optimised_rf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print('Accuracy: {}'.format(accuracy))

Accuracy: 0.3893312101910828
